In [57]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from joblib import dump
from sklearn.feature_extraction import FeatureHasher
import matplotlib

matplotlib.use("Agg")  # falls irgendwo Plots erzeugt werden



# __file__ is not defined in interactive environments (like Jupyter).
# Try to use it when available, otherwise fall back to the current working directory.
try:
	base_dir = Path(__file__).resolve().parent
except NameError:
	base_dir = Path.cwd()

file_path = base_dir.parent / "data" / "1311dataset.csv"

df = pd.read_csv(file_path)

df.head()

# Sort by account and date first
df['date_post'] = pd.to_datetime(df['date_post'], format='%Y%m%d')
df = df.sort_values(["bank_account_uuid", "date_post"])



In [58]:
# Calculate rolling features
df['amount_mean_5'] = df.groupby('bank_account_uuid')['amount'] \
                        .transform(lambda x: x.rolling(5, min_periods=1).mean())
df['amount_std_5'] = df.groupby('bank_account_uuid')['amount'] \
                       .transform(lambda x: x.rolling(5, min_periods=1).std()).fillna(0)
df['amount_change'] = df.groupby('bank_account_uuid')['amount'].diff()
df['amount_change'] = df['amount_change'].fillna(0)
# ✅ Correct receiver change flag (no MultiIndex)


df['month'] = df['date_post'].dt.month
df['dayofweek'] = df['date_post'].dt.dayofweek
df["year"] = df['date_post'].dt.year
# Time delta since last transaction
# Abstand berechnen
df['time_since_last_tx'] = (
    df.groupby(['bank_account_uuid', 'ref_iban'])['date_post']
      .diff().dt.days
)
df.drop("date_post", axis=1, inplace=True)
# globalen Mittelwert über alle gültigen Werte berechnen

# NaN durch den Mittelwert ersetzen
df['time_since_last_tx'] = df['time_since_last_tx'].fillna(30)


print(df.columns)
#gib mir die anomalien aus
anomalies = df[df['anomaly_description'].notna()]
#wie viele unique values hat jede spalte
for col in df.columns:
    print(col, df[col].nunique())

Index(['bank_account_uuid', 'business_partner_name', 'amount', 'currency',
       'ref_name', 'ref_iban', 'ref_swift', 'ref_bank', 'paym_note',
       'trns_type', 'pay_method', 'channel', 'anomaly_description',
       'amount_mean_5', 'amount_std_5', 'amount_change', 'month', 'dayofweek',
       'year', 'time_since_last_tx'],
      dtype='object')
bank_account_uuid 9
business_partner_name 9
amount 1804
currency 1
ref_name 196
ref_iban 204
ref_swift 196
ref_bank 194
paym_note 1800
trns_type 1
pay_method 3
channel 3
anomaly_description 18
amount_mean_5 1806
amount_std_5 1798
amount_change 1797
month 12
dayofweek 7
year 3
time_since_last_tx 22


In [59]:
df["anomaly_description"].value_counts()
df.head()

,bank_account_uuid,business_partner_name,amount,currency,ref_name,ref_iban,ref_swift,ref_bank,paym_note,trns_type,pay_method,channel,anomaly_description,amount_mean_5,amount_std_5,amount_change,month,dayofweek,year,time_since_last_tx
1009,1C447D4F1599450AB2AB4DFC163B529F,White Group,42516.45,USD,Ray Inc,GB33XJMF87452642213857,VCKQGB1T3DI,GM,New Equipment Purchase - PO#5858,DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,42516.450000,0.000000,0.00,1,3,2022,30.0
1077,1C447D4F1599450AB2AB4DFC163B529F,White Group,34306.00,USD,Thompson-Perez,GB24CEYO61531523398575,SUPQGBRSBLR,HT,Legal Fees - PO#9502,DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,38411.225000,5805.664872,-8210.45,1,4,2022,30.0
269,1C447D4F1599450AB2AB4DFC163B529F,White Group,2170.90,USD,Butler LLC Logistics,GB24PNVP62636341275547,KHFOGBIVO5J,BY,Payment for INV-2022-01-1567 (Term 15d),DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,26331.116667,21322.287033,-32135.10,1,0,2022,30.0
1766,1C447D4F1599450AB2AB4DFC163B529F,White Group,3039.55,USD,Perry Inc Logistics,GB41ZIWZ00768587878792,ZSYAGBWNT1E,GN,Payment for INV-2022-01-3885 (Term 15d),DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,20508.225000,20945.585510,868.65,2,3,2022,30.0
61,1C447D4F1599450AB2AB4DFC163B529F,White Group,18408.99,USD,Lopez Ltd,GB36UFFR27318410616772,WEBDGBJHZGS,KI,Legal Fees - PO#8377,DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,20088.378000,18163.686899,15369.44,2,6,2022,30.0


In [60]:
# columns_to_drop = ["bank_account_uuid", "ref_bank", "currency", "trns_type"]

# df = df.drop(columns=columns_to_drop)

In [61]:
df.head()

,bank_account_uuid,business_partner_name,amount,currency,ref_name,ref_iban,ref_swift,ref_bank,paym_note,trns_type,pay_method,channel,anomaly_description,amount_mean_5,amount_std_5,amount_change,month,dayofweek,year,time_since_last_tx
1009,1C447D4F1599450AB2AB4DFC163B529F,White Group,42516.45,USD,Ray Inc,GB33XJMF87452642213857,VCKQGB1T3DI,GM,New Equipment Purchase - PO#5858,DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,42516.450000,0.000000,0.00,1,3,2022,30.0
1077,1C447D4F1599450AB2AB4DFC163B529F,White Group,34306.00,USD,Thompson-Perez,GB24CEYO61531523398575,SUPQGBRSBLR,HT,Legal Fees - PO#9502,DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,38411.225000,5805.664872,-8210.45,1,4,2022,30.0
269,1C447D4F1599450AB2AB4DFC163B529F,White Group,2170.90,USD,Butler LLC Logistics,GB24PNVP62636341275547,KHFOGBIVO5J,BY,Payment for INV-2022-01-1567 (Term 15d),DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,26331.116667,21322.287033,-32135.10,1,0,2022,30.0
1766,1C447D4F1599450AB2AB4DFC163B529F,White Group,3039.55,USD,Perry Inc Logistics,GB41ZIWZ00768587878792,ZSYAGBWNT1E,GN,Payment for INV-2022-01-3885 (Term 15d),DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,20508.225000,20945.585510,868.65,2,3,2022,30.0
61,1C447D4F1599450AB2AB4DFC163B529F,White Group,18408.99,USD,Lopez Ltd,GB36UFFR27318410616772,WEBDGBJHZGS,KI,Legal Fees - PO#8377,DEBIT,WIRE,ONLINE_BANKING_PORTAL,NaN,20088.378000,18163.686899,15369.44,2,6,2022,30.0


In [62]:
#alle ausgeben lasse wo anomaly != nan ist
df.columns

Index(['bank_account_uuid', 'business_partner_name', 'amount', 'currency',
       'ref_name', 'ref_iban', 'ref_swift', 'ref_bank', 'paym_note',
       'trns_type', 'pay_method', 'channel', 'anomaly_description',
       'amount_mean_5', 'amount_std_5', 'amount_change', 'month', 'dayofweek',
       'year', 'time_since_last_tx'],
      dtype='object')

In [63]:
#wie viele unique values hat jede spalte
for col in df.columns:
    print(col, df[col].nunique())

bank_account_uuid 9
business_partner_name 9
amount 1804
currency 1
ref_name 196
ref_iban 204
ref_swift 196
ref_bank 194
paym_note 1800
trns_type 1
pay_method 3
channel 3
anomaly_description 18
amount_mean_5 1806
amount_std_5 1798
amount_change 1797
month 12
dayofweek 7
year 3
time_since_last_tx 22


In [ ]:
import pandas as pd

####!!!!!!

#seperat für test und traing aufrufen !!!!!!



#this method creates a new column 'valid_ref' that indicates whether the reference information (in the combo columns) are valid or not. They appear always together e.g a ref_name has always the same ref_iban, ref_swift, pay_method, channel, currency, trns_type.
#if one of the combo cols is wrong -> valid_ref = 0 ->  anomaly

df['account_partner_id'] = df['bank_account_uuid'].astype(str) + "_" + df['business_partner_name'].astype(str)
df = df.drop(columns=['bank_account_uuid', 'business_partner_name'])

combo_cols = ["ref_name", "ref_iban", "ref_swift", "pay_method", "channel", "currency", "trns_type"]

for col in range(len(combo_cols)):
    combo_observed = combo_cols[:col+1]
    combo_counts = (
        df.groupby(combo_observed, dropna=False)
          .size()
          .reset_index(name=f'combination_freq_{col+1}')
    )
    df = df.merge(combo_counts, on=combo_observed, how='left')

k = len(combo_cols)
freq_cols = [f'combination_freq_{i}' for i in range(1, k + 1)]

all_equal = (df[freq_cols].nunique(axis=1, dropna=False) == 1)

at_least_one_one = (df[freq_cols] == 1).any(axis=1)

df['valid_ref'] = (
    all_equal |
    (~all_equal & ~at_least_one_one)
).astype(int)

#drop combo columns
df = df.drop(columns=combo_cols + freq_cols)
df = df.drop(columns = ["ref_banks"])

In [65]:
df["anomaly_description"].head(10)


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
Name: anomaly_description, dtype: object

In [66]:
#gib mir die anomalien aus
anomalies = df[df['anomaly_description'].notna()]
anomalies.head()

,amount,ref_bank,paym_note,anomaly_description,amount_mean_5,amount_std_5,amount_change,month,dayofweek,year,time_since_last_tx,account_partner_id,valid_ref
57,194423.50,BH,Inv. #INV-2022-08-4870 (Term 45d),IBAN_MISMATCH_ANOMALY: IBAN changed from GB82T...,46458.912,83579.422960,192286.37,9,0,2022,30.0,1C447D4F1599450AB2AB4DFC163B529F_White Group,0
78,219079.28,KE,Payment for INV-2022-11-4873 (Term 45d),IBAN_MISMATCH_ANOMALY: IBAN changed from GB82T...,45994.286,96759.401133,216839.57,12,4,2022,30.0,1C447D4F1599450AB2AB4DFC163B529F_White Group,0
168,282021.00,BW,Invoice #INV-2023-11-4885 (Term 45d),IBAN_MISMATCH_ANOMALY: IBAN changed from GB82T...,58488.018,124961.689879,280068.09,12,0,2023,30.0,1C447D4F1599450AB2AB4DFC163B529F_White Group,0
259,79177.62,FM,Invoice #INV-2022-05-4900 (Term 45d),REMITTANCE_MISMATCH: Duplicate invoice in note...,18038.084,34199.386653,78468.39,10,4,2022,31.0,3373255939A942E48C9B633A126A4FCD_Martinez Ltd,1
306,82292.23,TZ,Inv. #INV-2023-01-4908 (Term 45d),REMITTANCE_MISMATCH: Duplicate invoice in note...,21750.178,34618.986560,81687.90,3,2,2023,29.0,3373255939A942E48C9B633A126A4FCD_Martinez Ltd,1


In [67]:
# Annahme: df ist dein DataFrame

# Häufigkeit jeder paym_note über den gesamten Datensatz zählen
note_counts = df['paym_note'].value_counts()

# Neue Spalte: 1, wenn paym_note nur einmal vorkommt, sonst 0
df['is_unique_paym_note'] = df['paym_note'].map(lambda x: 1 if note_counts[x] == 1 else 0)

# Ergebnis prüfen
print(df[['paym_note', 'is_unique_paym_note']].head())


                                 paym_note  is_unique_paym_note
0         New Equipment Purchase - PO#5858                    1
1                     Legal Fees - PO#9502                    1
2  Payment for INV-2022-01-1567 (Term 15d)                    1
3  Payment for INV-2022-01-3885 (Term 15d)                    1
4                     Legal Fees - PO#8377                    0


In [ ]:
df[df["paym_note"] == "Legal Fees - PO#8377"][]

,amount,ref_bank,paym_note,anomaly_description,amount_mean_5,amount_std_5,amount_change,month,dayofweek,year,time_since_last_tx,account_partner_id,valid_ref,is_unique_paym_note
4,18408.99,KI,Legal Fees - PO#8377,NaN,20088.378,18163.686899,15369.44,2,6,2022,30.0,1C447D4F1599450AB2AB4DFC163B529F_White Group,1,0
1584,36221.84,MZ,Legal Fees - PO#8377,NaN,27691.606,39275.728689,31927.97,8,0,2023,30.0,DEDF2F9C3181416DBCD8DBD430B5CC6C_Carroll-Taylor,1,0
